# Initialization

In [1]:
!uv pip show google-adk google-genai google-cloud-aiplatform | grep Version

Version: 1.2.1
Version: 1.97.0
Version: 1.19.0


In [2]:
import vertexai
from vertexai.preview import reasoning_engines

PROJECT_ID = "hello-world-418507"
LOCATION = "us-central1"
STAGING_BUCKET = "gs://2025-wy-agentspace"

vertexai.init(
    project=PROJECT_ID,
    location=LOCATION,
    staging_bucket=STAGING_BUCKET,
)

In [3]:
import datetime
from zoneinfo import ZoneInfo
from google.adk.agents import Agent

def get_weather(city: str) -> dict:
    """Retrieves the current weather report for a specified city.

    Args:
        city (str): The name of the city for which to retrieve the weather report.

    Returns:
        dict: status and result or error msg.
    """
    if city.lower() == "new york":
        return {
            "status": "success",
            "report": (
                "The weather in New York is sunny with a temperature of 25 degrees"
                " Celsius (41 degrees Fahrenheit)."
            ),
        }
    else:
        return {
            "status": "error",
            "error_message": f"Weather information for '{city}' is not available.",
        }


def get_current_time(city: str) -> dict:
    """Returns the current time in a specified city.

    Args:
        city (str): The name of the city for which to retrieve the current time.

    Returns:
        dict: status and result or error msg.
    """

    if city.lower() == "new york":
        tz_identifier = "America/New_York"
    else:
        return {
            "status": "error",
            "error_message": (
                f"Sorry, I don't have timezone information for {city}."
            ),
        }

    tz = ZoneInfo(tz_identifier)
    now = datetime.datetime.now(tz)
    report = (
        f'The current time in {city} is {now.strftime("%Y-%m-%d %H:%M:%S %Z%z")}'
    )
    return {"status": "success", "report": report}


root_agent = Agent(
    name="weather_time_agent",
    model="gemini-2.0-flash-exp",
    description=(
        "Agent to answer questions about the time and weather in a city."
    ),
    instruction=(
        "I can answer your questions about the time and weather in a city."
    ),
    tools=[get_weather, get_current_time],
)

In [4]:

app = reasoning_engines.AdkApp(
    agent=root_agent,
    enable_tracing=True,
)

## Local test

In [9]:
session = await app.async_create_session(user_id="u_123")
session

Session(id='20806ce0-1496-413c-8c50-4691899d95fe', app_name='default-app-name', user_id='u_123', state={}, events=[], last_update_time=1749700287.8736682)

In [11]:
await app.async_list_sessions(user_id="u_123")


ListSessionsResponse(sessions=[Session(id='8f5539f1-e379-4b3f-8b05-0d4814539a42', app_name='default-app-name', user_id='u_123', state={}, events=[], last_update_time=1749700231.096045), Session(id='20806ce0-1496-413c-8c50-4691899d95fe', app_name='default-app-name', user_id='u_123', state={}, events=[], last_update_time=1749700287.8736682)])

In [12]:
session = await app.async_get_session(user_id="u_123", session_id=session.id)
session

Session(id='20806ce0-1496-413c-8c50-4691899d95fe', app_name='default-app-name', user_id='u_123', state={}, events=[], last_update_time=1749700287.8736682)

In [16]:
async for event in app.async_stream_query(
    user_id="u_123",
    session_id=session.id,
    message="whats the weather in new york",
):
    print("*"*100)
    print(event)

****************************************************************************************************
{'content': {'parts': [{'function_call': {'id': 'adk-57b04ca8-121f-48ed-bcb8-6d5fa0bcf97c', 'args': {'city': 'new york'}, 'name': 'get_weather'}}], 'role': 'model'}, 'usage_metadata': {'candidates_token_count': 6, 'candidates_tokens_details': [{'modality': <MediaModality.TEXT: 'TEXT'>, 'token_count': 6}], 'prompt_token_count': 302, 'prompt_tokens_details': [{'modality': <MediaModality.TEXT: 'TEXT'>, 'token_count': 302}], 'total_token_count': 308, 'traffic_type': <TrafficType.ON_DEMAND: 'ON_DEMAND'>}, 'invocation_id': 'e-3968432e-5dd1-429e-8648-3004379f8db0', 'author': 'weather_time_agent', 'actions': {'state_delta': {}, 'artifact_delta': {}, 'requested_auth_configs': {}}, 'long_running_tool_ids': set(), 'id': 'qgHNlhs2', 'timestamp': 1749700402.244068}
****************************************************************************************************
{'content': {'parts': [{'function_r

## Deploy

In [24]:
from vertexai import agent_engines

remote_app = agent_engines.create(
    display_name="my_test_agent",
    agent_engine=root_agent,
    requirements=[
        "google-cloud-aiplatform[adk,agent_engines]"   
    ]
)

Deploying google.adk.agents.Agent as an application.
Identified the following requirements: {'cloudpickle': '3.1.1', 'google-cloud-aiplatform': '1.97.0', 'pydantic': '2.11.5'}
The following requirements are missing: {'cloudpickle', 'pydantic'}
The following requirements are appended: {'pydantic==2.11.5', 'cloudpickle==3.1.1'}
The final list of requirements: ['google-cloud-aiplatform[adk,agent_engines]', 'pydantic==2.11.5', 'cloudpickle==3.1.1']


I0000 00:00:1749705483.636018  741707 fork_posix.cc:71] Other threads are currently calling into gRPC, skipping fork() handlers


Using bucket 2025-wy-agentspace
Wrote to gs://2025-wy-agentspace/agent_engine/agent_engine.pkl
Writing to gs://2025-wy-agentspace/agent_engine/requirements.txt
Creating in-memory tarfile of extra_packages
Writing to gs://2025-wy-agentspace/agent_engine/dependencies.tar.gz
Creating AgentEngine
Create AgentEngine backing LRO: projects/671247654914/locations/us-central1/reasoningEngines/8973536606745001984/operations/2943560681036709888
View progress and logs at https://console.cloud.google.com/logs/query?project=hello-world-418507
AgentEngine created. Resource name: projects/671247654914/locations/us-central1/reasoningEngines/8973536606745001984
To use this AgentEngine in another session:
agent_engine = vertexai.agent_engines.get('projects/671247654914/locations/us-central1/reasoningEngines/8973536606745001984')


## Test in new session

- Pre-deployed agent engine id: 3326620808347910144

In [1]:
from vertexai import agent_engines

live_app = agent_engines.get('projects/671247654914/locations/us-central1/reasoningEngines/8973536606745001984')
session = await live_app.async_create_session(user_id="u_123")
session

{'id': '5044274024968880128',
 'events': [],
 'userId': 'u_123',
 'state': {},
 'appName': '8973536606745001984',
 'lastUpdateTime': 1749778482.654987}

In [2]:
# Check available operations
# live_app.operation_schemas()

In [3]:
await live_app.async_list_sessions(user_id="u_123")

{'sessions': [{'events': [],
   'state': {},
   'id': '5044274024968880128',
   'userId': 'u_123',
   'appName': '8973536606745001984',
   'lastUpdateTime': 1749778482.654987},
  {'events': [],
   'userId': 'u_123',
   'state': {},
   'id': '2735475528499724288',
   'appName': '8973536606745001984',
   'lastUpdateTime': 1749718266.498853}]}

In [4]:
import json

request_json = json.dumps(
    {
        # "artifacts": [
        #     {
        #         "file_name": "test_file_name",
        #         "versions": [{"version": "v1", "data": "v1data"}],
        #     }
        # ],
        # "authorizations": {
        #     "test_user_id1": {"access_token": "test_access_token"},
        #     "test_user_id2": {"accessToken": "test-access-token"},
        # },
        "user_id": "u_123",
        "message": {
            "parts": [{"text": "whats the weather in new york?"}],
            "role": "user",
        },
    }
)
events = list(live_app.streaming_agent_run_with_events(request_json=request_json))
events

[{'events': [{'content': {'parts': [{'function_call': {'id': 'adk-1a0ca32c-02fe-45f6-86a2-88f327ccf1f4',
        'args': {'city': 'new york'},
        'name': 'get_weather'}}],
     'role': 'model'},
    'usage_metadata': {'candidates_token_count': 6,
     'candidates_tokens_details': [{'modality': 'TEXT', 'token_count': 6}],
     'prompt_token_count': 176,
     'prompt_tokens_details': [{'modality': 'TEXT', 'token_count': 176}],
     'total_token_count': 182,
     'traffic_type': 'ON_DEMAND'},
    'invocation_id': 'e-886dc803-09c8-4848-925c-5b17abd71111',
    'author': 'weather_time_agent',
    'actions': {'state_delta': {},
     'artifact_delta': {},
     'requested_auth_configs': {}},
    'long_running_tool_ids': [],
    'id': 'FW4DFxRM',
    'timestamp': 1749778490.051801}]},
 {'events': [{'content': {'parts': [{'function_response': {'id': 'adk-1a0ca32c-02fe-45f6-86a2-88f327ccf1f4',
        'name': 'get_weather',
        'response': {'status': 'success',
         'report': 'The wea

## Logging

```bash
resource.type="aiplatform.googleapis.com/ReasoningEngine"
resource.labels.location="us-central1"
resource.labels.reasoning_engine_id="2590282269272834048"
```

In [19]:
from google.cloud import logging
from datetime import datetime, timedelta
import pytz

def get_recent_logs(engine_id, minutes=5):
    """
    Retrieve logs from a specific Reasoning Engine for the last specified minutes.
    
    Args:
        engine_id (str): The ID of the Reasoning Engine
        minutes (int): Number of minutes to look back for logs
    
    Returns:
        List of log entries
    """
    # Initialize the logging client
    client = logging.Client()
    
    # Calculate the time range in Pacific Time
    vegas_tz = pytz.timezone('America/Los_Angeles')
    end_time = datetime.now(vegas_tz)
    start_time = end_time - timedelta(minutes=minutes)
    
    # Format the start timestamp for the query
    start_time_str = start_time.strftime('%Y-%m-%dT%H:%M:%S.%fZ')
    
    # Construct the filter string
    filter_str = (
        'resource.type="aiplatform.googleapis.com/ReasoningEngine" '
        'resource.labels.location="us-central1" '
        f'resource.labels.reasoning_engine_id="{engine_id}" '
        f'timestamp >= "{start_time_str}"'
    )
    
    # Create an iterator for the logs and convert to list
    entries = list(client.list_entries(filter_=filter_str))
    
    print(f"Found {len(entries)} log entries in the last {minutes} minutes")
    
    # Print just the payload for each log entry
    for i, entry in enumerate(entries, 1):
        print(f"\n--- Log Entry {i} ---")
        print(f"Payload: {entry.payload}")



In [ ]:
# Get logs for the last 15 minutes
get_recent_logs(engine_id="8817529100903120896", minutes=3)

## Registration

- Copy and paste, and run the command in terminal


In [ ]:
%%bash

export PROJECT_NUMBER="671247654914"
export LOCATION="us-central1"
export APP_ID="agentspace-dev_1744685873939""
export ADK_DEPLOYMENT_ID="8973536606745001984"
export DISPLAY_NAME="Weather Agent ADK1.2 CMD"
export DESCRIPTION="Weather agent for New York, United States."
export TOOL_DESCRIPTION="Use this tool when user ask about weather in New York only"
export REASONING_ENGINE="projects/${PROJECT_NUMBER}/locations/{LOCATION}/reasoningEngines/${ADK_DEPLOYMENT_ID}"


curl -X POST \
  -H "Authorization: Bearer $(gcloud auth print-access-token)" \
  -H "Content-Type: application/json" \
  -H "X-Goog-User-Project: ${PROJECT_NUMBER}" \
  "https://discoveryengine.googleapis.com/v1alpha/projects/${PROJECT_NUMBER}/locations/global/collections/default_collection/engines/${APP_ID}/assistants/default_assistant/agents" \
  -d "{
    \"displayName\": \"${DISPLAY_NAME}\",
    \"description\": \"${DESCRIPTION}\",
    \"adk_agent_definition\": {
      \"tool_settings\": {
        \"tool_description\": \"${TOOL_DESCRIPTION}\"
      },
      \"provisioned_reasoning_engine\": {
        \"reasoning_engine\": \"projects/${PROJECT_NUMBER}/locations/${LOCATION}/reasoningEngines/${ADK_DEPLOYMENT_ID}\"
      }
    }
  }"


## Delete Agent Engine

In [ ]:
remote_app.delete(force=True)